<a href="https://colab.research.google.com/github/singhraj00/langchain-tutorial/blob/main/Langchain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chains in LangChain

### **Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.**

In [6]:
!pip install langchain
!pip install langchain_core langchain_groq
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00


## A Simple Chain Example

In [7]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

API_KEY = userdata.get('GROQ_API_KEY')

## call llm
llm = ChatGroq(model_name="llama-3.3-70b-versatile",api_key=API_KEY,temperature=0)

# create prompt
prompt = PromptTemplate(
    template="generate 5 interesting facts about {topic}",
    input_variable=['topic']
)

## create chain
chain = prompt | llm | StrOutputParser()

# invoke model
result = chain.invoke({'topic':'black hole'})

# print result
print(result)

# chain visulization
chain.get_graph().print_ascii()

Here are 5 interesting facts about black holes:

1. **Black Holes are Not Actually Black**: Despite their name, black holes are not actually black. They are regions in space where the gravitational pull is so strong that nothing, including light, can escape. However, the accretion disk of hot, swirling gas around a black hole can emit intense radiation, making it visible to telescopes. This radiation can be so bright that it outshines the entire galaxy that the black hole is located in.

2. **Black Holes Come in Different Sizes**: Black holes are often thought of as massive, cosmic monsters, but they actually come in a range of sizes. Stellar black holes are formed when a star collapses in on itself and are relatively small, with masses between 1.4 and 20 times that of the sun. Supermassive black holes, on the other hand, are found at the centers of galaxies and can have masses billions or even trillions of times that of the sun. There are also intermediate-mass black holes, which have

## we builds a little bit complex chain

In [11]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

API_KEY = userdata.get('GROQ_API_KEY')

## call llm
llm = ChatGroq(model_name="llama-3.3-70b-versatile",api_key=API_KEY,temperature=0)

prompt1 = PromptTemplate(
    template='generate a detailed report on the following topic: {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template="generate a 5 point summary from the following text \n {text}",
    input_variables = ['text']
)

parser = StrOutputParser()

## create chain
chain = prompt1 | llm | parser | prompt2 | llm | parser

result = chain.invoke({'topic':'Unemployment in India'})

print(result)

## visulaize chain

chain.get_graph().print_ascii()

Here is a 5-point summary of the report on unemployment in India:

1. **High Unemployment Rate**: The unemployment rate in India stood at 7.2% in 2020-21, with a significant increase from 5.3% in 2017-18, and is higher among the youth, with 17.4% of individuals in the 15-29 age group being unemployed.

2. **Regional Disparities**: Unemployment rates vary significantly across different states in India, with Haryana, Rajasthan, and Jammu and Kashmir having the highest unemployment rates, while Gujarat, Maharashtra, and Tamil Nadu have the lowest unemployment rates.

3. **Causes of Unemployment**: The high unemployment rate in India is attributed to several factors, including a large and growing population, lack of job creation, agricultural distress, skill mismatch, and a significant proportion of the workforce being employed in the informal sector.

4. **Consequences of Unemployment**: The high unemployment rate in India has significant consequences, including poverty and inequality, so

## In above code, we build an chain for an llm first prompt will generate an detailoed replort on following topics and second prompt will be genrate an 5 point summary from topics.

## Build an parallel chain

In [16]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

API_KEY = userdata.get('GROQ_API_KEY')

## we have used two different model
model_1 = ChatGroq(model_name="llama-3.3-70b-versatile",api_key=API_KEY,temperature=0)
model_2 = ChatGroq(model_name="llama-3.1-8b-instant",api_key=API_KEY,temperature=0)


prompt1 = PromptTemplate(
    template='generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='generate 5 short question answer from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='merge the provided notes and quiz into the following into a single documents \n notes -> {notes} and quiz -> {quiz}',
    input_variables = ['notes','quiz']
)

parser = StrOutputParser()

## create a parallel chain

parallel_chain = RunnableParallel({
    'notes': prompt1 | model_1 | parser,
     'quiz' : prompt2 | model_2 | parser
})

merge_chain = prompt3 | model_2 | parser

chain = parallel_chain | merge_chain

text = """
A Random Forest is a collection of decision trees that work together to make predictions. In this article, we'll explain how the Random Forest algorithm works and how to use it.

Understanding Intuition for Random Forest Algorithm
Random Forest algorithm is a powerful tree learning technique in Machine Learning to make predictions and then we do voting of all the tress to make prediction. They are widely used for classification and regression task.

It is a type of classifier that uses many decision trees to make predictions.
It takes different random parts of the dataset to train each tree and then it combines the results by averaging them. This approach helps improve the accuracy of predictions. Random Forest is based on ensemble learning.
Imagine asking a group of friends for advice on where to go for vacation. Each friend gives their recommendation based on their unique perspective and preferences (decision trees trained on different subsets of data). You then make your final decision by considering the majority opinion or averaging their suggestions (ensemble prediction).

Then - Multiple Decision Trees are created from the training data. Each tree is trained on a random subset of the data (with replacement) and a random subset of features. This process is known as bagging or bootstrap aggregating.
Each Decision Tree in the ensemble learns to make predictions independently.
When presented with a new, unseen instance, each Decision Tree in the ensemble makes a prediction.
The final prediction is made by combining the predictions of all the Decision Trees. This is typically done through a majority vote (for classification) or averaging (for regression).

Key Features of Random Forest
Handles Missing Data: Automatically handles missing values during training, eliminating the need for manual imputation.
Algorithm ranks features based on their importance in making predictions offering valuable insights for feature selection and interpretability.
Scales Well with Large and Complex Data without significant performance degradation.
Algorithm is versatile and can be applied to both classification tasks (e.g., predicting categories) and regression tasks (e.g., predicting continuous values).
How Random Forest Algorithm Works?
The random Forest algorithm works in several steps:

Random Forest builds multiple decision trees using random samples of the data. Each tree is trained on a different subset of the data which makes each tree unique.
When creating each tree the algorithm randomly selects a subset of features or variables to split the data rather than using all available features at a time. This adds diversity to the trees.
Each decision tree in the forest makes a prediction based on the data it was trained on. When making final prediction random forest combines the results from all the trees.
For classification tasks the final prediction is decided by a majority vote. This means that the category predicted by most trees is the final prediction.
For regression tasks the final prediction is the average of the predictions from all the trees.
The randomness in data samples and feature selection helps to prevent the model from overfitting making the predictions more accurate and reliable.
Assumptions of Random Forest
Each tree makes its own decisions: Every tree in the forest makes its own predictions without relying on others.
Random parts of the data are used: Each tree is built using random samples and features to reduce mistakes.
Enough data is needed: Sufficient data ensures the trees are different and learn unique patterns and variety.
Different predictions improve accuracy: Combining the predictions from different trees leads to a more accurate final results.
"""
result = chain.invoke({"text":text})
print(result)

## visulizing chain

chain.get_graph().print_ascii()

**Random Forest: Notes and Quiz**

**What is Random Forest?**
- A collection of decision trees that work together to make predictions.
- Used for classification and regression tasks.

**How it Works**
1. Multiple decision trees are created from random subsets of data.
2. Each tree makes a prediction independently.
3. Final prediction is made by combining all tree predictions (majority vote or averaging).

**Key Features**
- Handles missing data
- Ranks feature importance
- Scales well with large data
- Versatile (classification and regression)

**Assumptions**
- Each tree makes its own decisions
- Random data samples are used
- Enough data is needed
- Different predictions improve accuracy

**Quiz**

1. **Q:** What is a Random Forest?
**A:** A Random Forest is a collection of decision trees that work together to make predictions.

2. **Q:** How does Random Forest handle missing data?
**A:** Random Forest handles missing data.

3. **Q:** What is the final prediction made by Random Fores

## Builds an conditional chain

In [25]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.schema.runnable import RunnableBranch,RunnableLambda
from typing import Literal

API_KEY = userdata.get('GROQ_API_KEY')

## call llm
llm = ChatGroq(model_name="llama-3.3-70b-versatile",api_key=API_KEY,temperature=0)

parser = StrOutputParser()

class Feedback(BaseModel):
  sentiment: Literal['positive','negative'] = Field(description='give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into positive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables = {'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | llm | parser2

result = classifier_chain.invoke({"feedback":"good battery backup"}).sentiment

# print(result)

prompt2 = PromptTemplate(
    template='write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt2 = PromptTemplate(
    template='write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='write an appropriate response to this negative feedback \n {feedback}',
)

branch_chain = RunnableBranch(
    ## if else conditional chain
    (lambda x: x.sentiment == 'positive',prompt2 | llm | parser),
    (lambda x: x.sentiment == 'negative',prompt3 | llm | parser),
    RunnableLambda(lambda x:"could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback':'this is a terrible phone'}))

chain.get_graph().print_ascii()

"I'm sorry to hear that you're not satisfied. Can you please provide more details about your experience so I can better understand what went wrong? Your feedback is valuable to me, and I'll do my best to make things right and improve for the future."
    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *        